# Saving and Loading a Model Notebook

You can save and/or load an ESM using three different options:
1. **Workspace**: you can save and load an ESM to and from your workspace.
2. **MLflow Run**: you can load an ESM from an MLflow run.
3. **MLflow Model Registry**: if you are working on Databricks, you can register and load an ESM to and from the MLflow Model Registry.

# Check Environment Variables
Before installing Hybrid Intelligence in the notebook you need to set these Environment Variables externally as described in the User Guide https://docs.umnai.com/set-up-your-environment. 
This section checks that the environment variables have been set correctly and throws an error if not.

In [1]:
import os

umnai_env_vars = {
    'UMNAI_CLIENT_ID',
    'UMNAI_CLIENT_SECRET',
    'PIP_EXTRA_INDEX_URL',
}

if not umnai_env_vars.issubset(os.environ.keys()):
    raise ValueError(
        'UMNAI environment variables not set correctly. They need to be set before using the Umnai library.'
    )

# Install Hybrid Intelligence
Next we install the UMNAI Platform.

In [2]:
%pip install umnai-platform --quiet


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


# Set Workspace Paths
Now we will set the the experiment name and workspace paths.

In [3]:
EXP_NAME = 'saveloadESM_adult_income'
WS_PATH = 'resources/workspaces/'+EXP_NAME
EXP_PATH = EXP_NAME

# Import and Prepare Dataset
Import the dataset to a Pandas DataFrame and the clean data in preparation for onboarding into Hybrid Intelligence.

In [4]:
import pandas as pd
import numpy as np

# Import Adult Income Dataset to pandas dataframe: 
# This dataset can be downloaded from https://archive.ics.uci.edu/dataset/2/adult 
column_names = ["Age", "WorkClass", "fnlwgt", "Education", "EducationNum", "MaritalStatus", "Occupation", "Relationship", "Race", "Gender", "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"]
dataset_df = pd.read_csv('https://raw.githubusercontent.com/umnaibase/umnai-examples/main/data/adult.data', names = column_names)

# Data Preparation:
dataset_df = dataset_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)    # Remove whitespaces
dataset_df["Income"] = np.where((dataset_df["Income"] == '<=50K'), 0, 1)                # Replace Target values with [0,1]
dataset_df.tail(5)

,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,1


# Create or Open a Hybrid Intelligence Workspace
Workspaces are used by the Hybrid Intelligence framework to organize your data and models together in one place.

In [5]:
from umnai.workspaces.context import Workspace

# Open a workspace
ws = Workspace.open(
    path = WS_PATH,
    experiment = EXP_PATH
)

ws # Prints workspace details to confirm created/opened

# Onboard Hybrid Intelligence Dataset

Onboard the Pandas DataFrame into a Hybrid Intelligence dataset.

In [6]:
from umnai.data.datasets import Dataset
from umnai.data.enums import PredictionType

dataset = Dataset.from_pandas(
    dataset_df,
    prediction_type=PredictionType.CLASSIFICATION,
    features=list(dataset_df.drop(['Income'], axis=1).columns),    # All columns except 'Income' are features
    targets=['Income'],
)

dataset # Prints dataset details to confirm created/opened

MLFLOW Run ID: 497fbf56c39b4252acdfac3965d3ef97:   0%|          | 0/36 [00:00<?, ?it/s]

Processing Tasks:   0%|          | 0/15 [00:00<?, ?it/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Processing Tasks:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: /opt/atlassian/pipelines/agent/build/demo-notebooks/resources/workspaces/saveloadESM_adult_income/preprocessing/dataset_name=Dataset_8dd343fa/assets


Dataset(id=eb4ae152-79e8-4d16-af79-e6a1c75d22d8; name=Dataset_8dd343fa; is_named=False; workspace_id=None)

# Induce a Hybrid Intelligence Model

First create the ModelInducer to set up the induction parameters and settings. Then simply use the ModelInducer to induce the ESM from the onboarded dataset.

In [7]:
from umnai.induction.inducer import ModelInducer
from umnai.esm.model import ESM

# Induce a simple model quickly using fast execution parameters
model_inducer = ModelInducer(
    max_interactions=3,
    max_interaction_degree=2,
    max_polynomial_degree=2,
    trials=2,
    estimators=2,
    batch_size=512,
    iterations=2,
)

# # Induce a more realistic model using default Induction parameters:
# model_inducer = ModelInducer()

# Create an ESM using Induction
esm = model_inducer.induce(dataset)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 13:24:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


21/21 [==============================] - 0s 2ms/step


[Modules] - MLFLOW Run ID: c5d60526d51c412b89631b6d258a20ff:   0%|          | 0/25 [00:00<?, ?it/s]

21/21 [==============================] - 0s 1ms/step


21/21 [==============================] - 0s 1ms/step


...

2024/03/01 13:28:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: /tmp/tmp3ailo9ou/model/data/model/assets


/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Note ESM ID and MLflow run ID
Note down:
- ESM ID: needed to load a model from a workspace.
- MLflow run ID: needed to load a model from an MLflow run.

In [8]:
# Save ESM ID and MLFLow Run ID for use in this notebook session
MLflow_Run_ID = esm.producer_run_id
ESM_ID = esm.id

# Note ESM ID and MLFLow Run ID for use in another notebook or session
print("MLflow Run ID: ", esm.producer_run_id)
print("ESM ID: ", esm.id)

MLflow Run ID:  c5d60526d51c412b89631b6d258a20ff
ESM ID:  Dataset_8dd343fa_c5d60526d51c412b89631b6d258a20ff


# Note ESM ID and MLflow run ID
Note down:
- ESM ID: needed to load a model from a workspace.
- MLflow run ID: needed to load a model from an MLflow run.

In [9]:
# Save ESM ID and MLFLow Run ID for use in this notebook session
MLflow_Run_ID = esm.producer_run_id
ESM_ID = esm.id

# Note ESM ID and MLFLow Run ID for use in another notebook or session
print("MLflow Run ID: ", esm.producer_run_id)
print("ESM ID: ", esm.id)

MLflow Run ID:  c5d60526d51c412b89631b6d258a20ff
ESM ID:  Dataset_8dd343fa_c5d60526d51c412b89631b6d258a20ff


# Model Save Options

## Save a model to a workspace

In [10]:
# Save the ESM to your workspace
esm.save_to_workspace()

INFO:tensorflow:Assets written to: /opt/atlassian/pipelines/agent/build/demo-notebooks/resources/workspaces/saveloadESM_adult_income/models/Dataset_8dd343fa_c5d60526d51c412b89631b6d258a20ff/assets


## Register a Model on MLflow Registry

You can register an ESM on the MLflow Model Registry using `register_mlflow()` and passing the desired ESM name. If the `name` is unspecified, the experiment run name will be used. If a model with the same name is already registered, then a new version will be created.

In [11]:
# Register ESM on MLflow Registry
esm.register_mlflow(name='AdultIncome')

Successfully registered model 'AdultIncome'.
Created version '1' of model 'AdultIncome'.


<ModelVersion: aliases=[], creation_timestamp=1709299716917, current_stage='None', description=None, last_updated_timestamp=1709299716917, name='AdultIncome', run_id='c5d60526d51c412b89631b6d258a20ff', run_link=None, source='/tmp/mlruns/217313284977281485/c5d60526d51c412b89631b6d258a20ff/artifacts/umnai_models/models', status='READY', status_message=None, tags={}, user_id=None, version=1>

# Model Load Options

## Load a model from a workspace

To load an ESM from your workspace, you will need the ESM ID.

In [12]:
# Use the ESM_ID previously obtained after the induction step above
esm_ws = ESM.from_workspace(id=ESM_ID)

# Check ESM was loaded by comparing its ID
if esm_ws.id == ESM_ID: 
    print("\nLoad from Workspace SUCCESSFUL.")
else:
    print("\nLoad from Workspace NOT SUCCESSFUL.")


Load from Workspace SUCCESSFUL.


## Load a model from an MLflow Run

To load an ESM from an MLflow Run, you will need the MLflow Run ID.

In [13]:
# Use the MLflow_Run_ID previously obtained after the induction step above
esm_mlrun = ESM.from_mlflow_run(run_id=MLflow_Run_ID)

# Check ESM was loaded by comparing its ID
if esm_mlrun.id == ESM_ID: 
    print("\nLoad from MLflow run SUCCESSFUL.")
else:
    print("\nLoad from MLflow run NOT SUCCESSFUL.")


Load from MLflow run SUCCESSFUL.


## Load a model from the MLflow Registry 

**NOTE:** MLflow Registry is on only supported on Databricks. This cell will only run on Databricks.

You can load an ESM from the MLflow Model Registry using `from_mlflow_registry()` and passing the desired model name and version. 

The code below automatically detects the latest model version and loads it.

In [14]:
import mlflow
import os
from mlflow import MlflowClient

# Get the latest version name of a registered model.
client = MlflowClient()
# Get all model versions
model_versions = client.search_registered_models(filter_string="name='AdultIncome'")
# Find the latest version
latest_version = str(max(model_versions[0].latest_versions, key = lambda v: v.version).version)

# Load latest version
esm_mlreg = ESM.from_mlflow_registry(model_name='AdultIncome', model_version=latest_version)

# Check ESM was loaded by comparing its ID
if esm_mlreg.id == ESM_ID:
    print("\nLoad from MLflow Registry SUCCESSFUL.")
else:
    print("\nLoad from MLflow Registry NOT SUCCESSFUL.")



Load from MLflow Registry SUCCESSFUL.
